<a href="https://colab.research.google.com/github/james130625/notebook/blob/main/13_NASDAQ_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요한 패키지 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm -rf ~/.cache/matplotlib

# FinanceDataReader 설치
!pip install finance-datareader

# matplotlib, pandas 임포트 및 주피터 출력 설정, 한글 폰트 지정
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
from datetime import datetime

plt.rcParams['font.family'] = 'NanumBarunGothic'  # 한글 출력을 위한 폰트 설정

# NASDAQ 상장 목록 가져오기
nasdaq_list = fdr.StockListing('NASDAQ')

# 데이터프레임의 열 이름 확인
print(nasdaq_list.columns)

# 열 이름 확인 후 'Symbol' 열 사용
symbol_column_name = 'Symbol'
close_column_name = 'Close'

# 필터링 조건
price_lower = 5
price_upper = 10

# 필터링된 데이터프레임 저장용 리스트
filtered_stocks_data = []

# 오늘 날짜 구하기
today = datetime.today().strftime('%Y-%m-%d')

# 각 종목의 데이터를 가져와서 조건에 맞는지 확인 및 상태 식별
for symbol in nasdaq_list[symbol_column_name]:
    try:
        # 종목 데이터 불러오기 (2023-01-02부터 오늘까지)
        data = fdr.DataReader(symbol, '2023-01-02', today)

        # 종가가 5-10 달러 사이에 있는지 확인
        if data[close_column_name].iloc[-1] < price_lower or data[close_column_name].iloc[-1] > price_upper:
            continue

        # 이동 평균 계산
        data['SMA20'] = data[close_column_name].rolling(window=20).mean()
        data['SMA50'] = data[close_column_name].rolling(window=50).mean()
        data['SMA200'] = data[close_column_name].rolling(window=200).mean()

        # 거래량 평균 계산
        avg_volume = data['Volume'].mean()

        # 변동폭 계산 (종가의 표준편차)
        volatility = data[close_column_name].std()

        # MACD 및 시그널 계산
        exp1 = data[close_column_name].ewm(span=12, adjust=False).mean()
        exp2 = data[close_column_name].ewm(span=26, adjust=False).mean()
        macd = exp1 - exp2
        signal = macd.ewm(span=9, adjust=False).mean()

        # 가장 최근 종가
        recent_close = data[close_column_name].iloc[-1]

        # 추세 계산
        trend = 'Uptrend' if macd.iloc[-1] > signal.iloc[-1] else 'Downtrend'

        # 매수가 범위 계산 (±2%)
        buy_range_low = recent_close * 0.98
        buy_range_high = recent_close * 1.02

        # 1차, 2차, 3차 매수가
        first_buy = buy_range_low
        second_buy = (buy_range_low + buy_range_high) / 2
        third_buy = buy_range_high

        # 익절가 (최근 종가의 +10%)
        take_profit = recent_close * 1.10

        # 손절가 (최근 종가의 -5%)
        stop_loss = recent_close * 0.95

        # 상태 식별
        data['Phase'] = 'Sleeping'
        conditions = [
            (data['SMA20'] > data['SMA50']) & (data['SMA50'] > data['SMA200']),
            (data['SMA20'] > data['SMA50']) & (data['SMA50'] < data['SMA200']),
        ]
        choices = ['Eating', 'Awakening']
        data['Phase'] = np.select(conditions, choices, default='Sleeping')

        # 조건 필터링
        if macd.iloc[-1] > signal.iloc[-1]:  # MACD 매수 시그널
            filtered_stocks_data.append({
                'Symbol': symbol,
                'Current Price': recent_close,
                'Recent Close': recent_close,
                'Trend': trend,
                'Buy Range': f"{buy_range_low:.2f} - {buy_range_high:.2f}",
                'First Buy': first_buy,
                'Second Buy': second_buy,
                'Third Buy': third_buy,
                'Take Profit': take_profit,
                'Stop Loss': stop_loss,
                'Avg Volume': avg_volume,
                'Volatility': volatility,
                'Phase': data['Phase'].iloc[-1]
            })
    except Exception as e:
        # 데이터 불러오기에 실패하면 무시
        print(f"Failed to process {symbol}: {e}")
        continue

# 필터링된 종목 데이터프레임 생성 및 정렬
filtered_stocks_df = pd.DataFrame(filtered_stocks_data)

# 거래량이 많고 변동폭이 낮은 순으로 정렬
filtered_stocks_df = filtered_stocks_df.sort_values(by=['Avg Volume', 'Volatility'], ascending=[False, True])

# 상위 10개 종목 선정
top_10_stocks_df = filtered_stocks_df.head(10)

# 결과 출력
print("조건에 맞는 상위 10개 종목 목록:")
print(top_10_stocks_df)

# 필터링된 종목 리스트 데이터프레임 출력 함수
def display_dataframe_to_user(name: str, dataframe: pd.DataFrame) -> None:
    from IPython.display import display, HTML
    display(HTML(f"<h2>{name}</h2>"))
    display(dataframe)

display_dataframe_to_user(name="조건에 맞는 상위 10개 NASDAQ 종목 목록", dataframe=top_10_stocks_df)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (5,612 kB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 121913 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr

100%|██████████| 3765/3765 [00:19<00:00, 192.90it/s]


Index(['Symbol', 'Name', 'IndustryCode', 'Industry'], dtype='object')
HTTP Error 404: Not Found  - symbol "GWACU" not found or invalid periods
Failed to process GWACU: 'Close'
HTTP Error 429: Too Many Requests  - symbol "CGEN" not found or invalid periods
Failed to process CGEN: 'Close'
HTTP Error 429: Too Many Requests  - symbol "ITI" not found or invalid periods
Failed to process ITI: 'Close'
HTTP Error 429: Too Many Requests  - symbol "SANG" not found or invalid periods
Failed to process SANG: 'Close'
HTTP Error 429: Too Many Requests  - symbol "GLSI" not found or invalid periods
Failed to process GLSI: 'Close'
HTTP Error 429: Too Many Requests  - symbol "CLLS" not found or invalid periods
Failed to process CLLS: 'Close'
HTTP Error 429: Too Many Requests  - symbol "BAER" not found or invalid periods
Failed to process BAER: 'Close'
HTTP Error 429: Too Many Requests  - symbol "FFNW" not found or invalid periods
Failed to process FFNW: 'Close'
HTTP Error 429: Too Many Requests  - symbo

,Symbol,Current Price,Recent Close,Trend,Buy Range,First Buy,Second Buy,Third Buy,Take Profit,Stop Loss,Avg Volume,Volatility,Phase
2,SOFI,6.99,6.99,Uptrend,6.85 - 7.13,6.8502,6.99,7.1298,7.689,6.6405,4.359280e+07,1.304543,Sleeping
0,WBD,8.27,8.27,Uptrend,8.10 - 8.44,8.1046,8.27,8.4354,9.097,7.8565,2.311970e+07,2.222927,Sleeping
10,JBLU,5.56,5.56,Uptrend,5.45 - 5.67,5.4488,5.56,5.6712,6.116,5.2820,1.371437e+07,1.343962,Sleeping
1,ERIC,6.22,6.22,Uptrend,6.10 - 6.34,6.0956,6.22,6.3444,6.842,5.9090,1.197217e+07,0.438418,Eating
17,CORZ,7.78,7.78,Uptrend,7.62 - 7.94,7.6244,7.78,7.9356,8.558,7.3910,4.792725e+06,0.924105,Sleeping
26,LESL,5.12,5.12,Uptrend,5.02 - 5.22,5.0176,5.12,5.2224,5.632,4.8640,3.742974e+06,2.991279,Awakening
6,ASTS,8.62,8.62,Uptrend,8.45 - 8.79,8.4476,8.62,8.7924,9.482,8.1890,3.541542e+06,1.372631,Awakening
20,SHLS,6.81,6.81,Uptrend,6.67 - 6.95,6.6738,6.81,6.9462,7.491,6.4695,3.470951e+06,6.020324,Sleeping
28,HUT,9.68,9.68,Uptrend,9.49 - 9.87,9.4864,9.68,9.8736,10.648,9.1960,3.236508e+06,2.908045,Awakening
56,BYND,7.59,7.59,Uptrend,7.44 - 7.74,7.4382,7.59,7.7418,8.349,7.2105,2.881674e+06,3.800554,Awakening


필요한 패키지 설치

In [2]:
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm -rf ~/.cache/matplotlib

# FinanceDataReader 설치
!pip install finance-datareader


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

라이브러리 임포트 및 폰트 설정

In [3]:
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
from datetime import datetime

plt.rcParams['font.family'] = 'NanumBarunGothic'  # 한글 출력을 위한 폰트 설정


NASDAQ 상장 목록 가져오기

In [4]:
nasdaq_list = fdr.StockListing('NASDAQ')

# 데이터프레임의 열 이름 확인
print(nasdaq_list.columns)

# 열 이름 확인 후 'Symbol' 열 사용
symbol_column_name = 'Symbol'
close_column_name = 'Close'

# 필터링 조건
price_lower = 5
price_upper = 10

# 필터링된 데이터프레임 저장용 리스트
filtered_stocks_data = []

# 오늘 날짜 구하기
today = datetime.today().strftime('%Y-%m-%d')


100%|██████████| 3765/3765 [00:18<00:00, 202.14it/s]

Index(['Symbol', 'Name', 'IndustryCode', 'Industry'], dtype='object')


각 종목의 데이터를 가져와서 조건에 맞는지 확인 및 상태 식별

In [5]:
for symbol in nasdaq_list[symbol_column_name]:
    try:
        # 종목 데이터 불러오기 (2023-01-02부터 오늘까지)
        data = fdr.DataReader(symbol, '2023-01-02', today)

        # 종가가 5-10 달러 사이에 있는지 확인
        if data[close_column_name].iloc[-1] < price_lower or data[close_column_name].iloc[-1] > price_upper:
            continue

        # 이동 평균 계산
        data['SMA20'] = data[close_column_name].rolling(window=20).mean()
        data['SMA50'] = data[close_column_name].rolling(window=50).mean()
        data['SMA200'] = data[close_column_name].rolling(window=200).mean()

        # 거래량 평균 계산
        avg_volume = data['Volume'].mean()

        # 변동폭 계산 (종가의 표준편차)
        volatility = data[close_column_name].std()

        # MACD 및 시그널 계산
        exp1 = data[close_column_name].ewm(span=12, adjust=False).mean()
        exp2 = data[close_column_name].ewm(span=26, adjust=False).mean()
        macd = exp1 - exp2
        signal = macd.ewm(span=9, adjust=False).mean()

        # 가장 최근 종가
        recent_close = data[close_column_name].iloc[-1]

        # 추세 계산
        trend = 'Uptrend' if macd.iloc[-1] > signal.iloc[-1] else 'Downtrend'

        # 매수가 범위 계산 (±2%)
        buy_range_low = recent_close * 0.98
        buy_range_high = recent_close * 1.02

        # 1차, 2차, 3차 매수가
        first_buy = buy_range_low
        second_buy = (buy_range_low + buy_range_high) / 2
        third_buy = buy_range_high

        # 익절가 (최근 종가의 +10%)
        take_profit = recent_close * 1.10

        # 손절가 (최근 종가의 -5%)
        stop_loss = recent_close * 0.95

        # 상태 식별
        data['Phase'] = 'Sleeping'
        conditions = [
            (data['SMA20'] > data['SMA50']) & (data['SMA50'] > data['SMA200']),
            (data['SMA20'] > data['SMA50']) & (data['SMA50'] < data['SMA200']),
        ]
        choices = ['Eating', 'Awakening']
        data['Phase'] = np.select(conditions, choices, default='Sleeping')

        # 조건 필터링
        if macd.iloc[-1] > signal.iloc[-1]:  # MACD 매수 시그널
            filtered_stocks_data.append({
                'Symbol': symbol,
                'Current Price': recent_close,
                'Recent Close': recent_close,
                'Trend': trend,
                'Buy Range': f"{buy_range_low:.2f} - {buy_range_high:.2f}",
                'First Buy': first_buy,
                'Second Buy': second_buy,
                'Third Buy': third_buy,
                'Take Profit': take_profit,
                'Stop Loss': stop_loss,
                'Avg Volume': avg_volume,
                'Volatility': volatility,
                'Phase': data['Phase'].iloc[-1]
            })
    except Exception as e:
        # 데이터 불러오기에 실패하면 무시
        print(f"Failed to process {symbol}: {e}")
        continue


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
HTTP Error 429: Too Many Requests  - symbol "HBNC" not found or invalid periods
Failed to process HBNC: 'Close'
HTTP Error 429: Too Many Requests  - symbol "TRST" not found or invalid periods
Failed to process TRST: 'Close'
HTTP Error 429: Too Many Requests  - symbol "FULC" not found or invalid periods
Failed to process FULC: 'Close'
HTTP Error 429: Too Many Requests  - symbol "RBBN" not found or invalid periods
Failed to process RBBN: 'Close'
HTTP Error 429: Too Many Requests  - symbol "CNSL" not found or invalid periods
Failed to process CNSL: 'Close'
HTTP Error 429: Too Many Requests  - symbol "FNKO" not found or invalid periods
Failed to process FNKO: 'Close'
HTTP Error 429: Too Many Requests  - symbol "MNMD" not found or invalid periods
Failed to process MNMD: 'Close'
HTTP Error 429: Too Many Requests  - symbol "GAIN" not found or invalid periods
Failed to process GAIN: 'Close'
HTTP Error 429: Too Many Requests  - symbol "CMPO" not found or inva

필터링된 종목 데이터프레임 생성 및 정렬

In [6]:
filtered_stocks_df = pd.DataFrame(filtered_stocks_data)

# 거래량이 많고 변동폭이 낮은 순으로 정렬
filtered_stocks_df = filtered_stocks_df.sort_values(by=['Avg Volume', 'Volatility'], ascending=[False, True])

# 상위 10개 종목 선정
top_10_stocks_df = filtered_stocks_df.head(10)

# 결과 출력
print("조건에 맞는 상위 10개 종목 목록:")
print(top_10_stocks_df)


조건에 맞는 상위 10개 종목 목록:
   Symbol  Current Price  Recent Close    Trend    Buy Range  First Buy  \
2    SOFI           6.99          6.99  Uptrend  6.85 - 7.13     6.8502   
0     WBD           8.27          8.27  Uptrend  8.10 - 8.44     8.1046   
10   JBLU           5.56          5.56  Uptrend  5.45 - 5.67     5.4488   
1    ERIC           6.22          6.22  Uptrend  6.10 - 6.34     6.0956   
6    ASTS           8.62          8.62  Uptrend  8.45 - 8.79     8.4476   
3    SMMT           7.72          7.72  Uptrend  7.57 - 7.87     7.5656   
12   VIAV           7.40          7.40  Uptrend  7.25 - 7.55     7.2520   
5     DLO           8.10          8.10  Uptrend  7.94 - 8.26     7.9380   
4    GDRX           8.55          8.55  Uptrend  8.38 - 8.72     8.3790   
14     LZ           8.41          8.41  Uptrend  8.24 - 8.58     8.2418   

    Second Buy  Third Buy  Take Profit  Stop Loss    Avg Volume  Volatility  \
2         6.99     7.1298        7.689     6.6405  4.359280e+07    1.30454

필터링된 종목 리스트 데이터프레임 출력 함수

In [7]:
def display_dataframe_to_user(name: str, dataframe: pd.DataFrame) -> None:
    from IPython.display import display, HTML
    display(HTML(f"<h2>{name}</h2>"))
    display(dataframe)

display_dataframe_to_user(name="조건에 맞는 상위 10개 NASDAQ 종목 목록", dataframe=top_10_stocks_df)


,Symbol,Current Price,Recent Close,Trend,Buy Range,First Buy,Second Buy,Third Buy,Take Profit,Stop Loss,Avg Volume,Volatility,Phase
2,SOFI,6.99,6.99,Uptrend,6.85 - 7.13,6.8502,6.99,7.1298,7.689,6.6405,4.359280e+07,1.304543,Sleeping
0,WBD,8.27,8.27,Uptrend,8.10 - 8.44,8.1046,8.27,8.4354,9.097,7.8565,2.311970e+07,2.222927,Sleeping
10,JBLU,5.56,5.56,Uptrend,5.45 - 5.67,5.4488,5.56,5.6712,6.116,5.2820,1.371437e+07,1.343962,Sleeping
1,ERIC,6.22,6.22,Uptrend,6.10 - 6.34,6.0956,6.22,6.3444,6.842,5.9090,1.197217e+07,0.438418,Eating
6,ASTS,8.62,8.62,Uptrend,8.45 - 8.79,8.4476,8.62,8.7924,9.482,8.1890,3.541542e+06,1.372631,Awakening
3,SMMT,7.72,7.72,Uptrend,7.57 - 7.87,7.5656,7.72,7.8744,8.492,7.3340,2.230660e+06,1.363717,Eating
12,VIAV,7.40,7.40,Uptrend,7.25 - 7.55,7.2520,7.40,7.5480,8.140,7.0300,1.893063e+06,1.190141,Sleeping
5,DLO,8.10,8.10,Uptrend,7.94 - 8.26,7.9380,8.10,8.2620,8.910,7.6950,1.576551e+06,2.673477,Sleeping
4,GDRX,8.55,8.55,Uptrend,8.38 - 8.72,8.3790,8.55,8.7210,9.405,8.1225,1.563161e+06,0.921441,Eating
14,LZ,8.41,8.41,Uptrend,8.24 - 8.58,8.2418,8.41,8.5782,9.251,7.9895,1.269549e+06,1.761326,Sleeping
